In [3]:
%pip install google.colab
from google.colab import files
uploaded = files.upload()

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement google.colab (from versions: none)
ERROR: No matching distribution found for google.colab

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'google'

In [ ]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
%pip install datasets
from datasets import Dataset
%pip install huggingface_hub
from huggingface_hub import login

#print(torch.cuda.is_available())  # Should return True if GPU is available
#print(torch.version.cuda)

In [ ]:
key = open("hugging_face_key.txt", "r").read()
login(token=key)

# This code loads in the model being used
model_name = "meta-llama/Llama-3.1-8B"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# This code loads in the fine-tuning dataset
formatted_data = []
for example in raw_data:
    text = f"Instruction: {example['instruction']}\nInput: {example['input']}\nOutput: {example['output']}"
    formatted_data.append({"text": text})
dataset = Dataset.from_list(formatted_data)

# This code tokenizes the dataset, which converts the text into tokens
def tokenize_function(examples):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
decoded_text = tokenizer.decode(tokenized_datasets[0]['input_ids'], skip_special_tokens=True)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

{'text': 'Instruction: Create a learning question based on the provided Braille\nInput: Conference Room A\nOutput: What did the Braille read? Option 1: Conference Room B, Option 2: Conference Room A, Option 3: Meeting Room C, Option 4: Hallway', 'input_ids': [6310, 2762, 25, 13610, 257, 4673, 1808, 1912, 319, 262, 2810, 9718, 8270, 198, 20560, 25, 8785, 10096, 317, 198, 26410, 25, 1867, 750, 262, 9718, 8270, 1100, 30, 16018, 352, 25, 8785, 10096, 347, 11, 16018, 362, 25, 8785, 10096, 317, 11, 16018, 513, 25, 22244, 10096, 327, 11, 16018, 604, 25, 4789, 1014, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5

In [ ]:
%pip install wandb
import wandb
wandb.init(anonymous="allow")

# This code sets up the configurations for QLoRA, which will be used for fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]
)

# This code loads in the Llama 3.1 large language model and applies the Parameter Efficient Fine-Tuning (PEFT) in using QLoRA
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,   # Ensure proper dtype for model
    device_map="auto",           # Automatically assign layers to GPU/CPU
)
model = get_peft_model(model, lora_config)

# This code sets up the arguments that will be referred to in training the model, and sets up the trainer based on this
training_args = TrainingArguments(
    output_dir="./finetuned_llm",
    evaluation_strategy="no",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=.0002
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets
)

# This code runs the actual model training and saves it so it can be used for future inference
trainer.train()
model.save_pretrained("./finetuned_llm")
tokenizer.save_pretrained("./finetuned_llm")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: anony-mouse-660117061587330063 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.